In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.2 MB/s 
     |████████████████████████████████| 5.8 MB 29.4 MB/s 
     |████████████████████████████████| 1.3 MB 63.1 MB/s 
     |████████████████████████████████| 182 kB 64.6 MB/s 
     |████████████████████████████████| 7.6 MB 65.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7283988a004a41cbd12d5f343d0f710f584500f691a0a65a7d42ce42edb1fd65
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 66.2 MB/s 
     |████████████████████████████████| 212 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator, CECorrelationEvaluator
from sentence_transformers.readers import InputExample

import datasets
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/cross-encode/

In [ ]:
train = datasets.load_dataset('stsb_multi_mt','fr', split='train')
test = datasets.load_dataset('stsb_multi_mt','fr', split='test')

### Create Input Examples

In [ ]:
train_samples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=float(row['similarity_score']/5.0)) for row in train] #similarity score
test_sample = [InputExample(texts=[row['sentence1'], row['sentence2']], label=float(row['similarity_score']/5.0)) for row in test]

### load model

In [ ]:
model = CrossEncoder('camembert-base', num_labels=1)
evaluator = CECorrelationEvaluator.from_input_examples(test_sample)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=8)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias

### Train model

In [ ]:
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=1,
          evaluation_steps=100,
          warmup_steps=50,
          output_path="./output-nli-stsb")

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

In [ ]:
# scores = model.predict([["My first", "sentence pair"],  
#                         ["Second text", "pair"]])

In [ ]:
# scores

array([0.22770123, 0.23233286], dtype=float32)

In [ ]:
"""
This example computes the score between a query and all possible
sentences in a corpus using a Cross-Encoder for semantic textual similarity (STS).
It output then the most similar sentences for the given query.
"""
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np

# Pre-trained cross encoder
# model = CrossEncoder('cross-encoder/stsb-distilroberta-base')
# model = CrossEncoder('cross-encoder/nli-deberta-v3-base')
model_saved = CrossEncoder('output-nli-stsb', max_length=512)

# We want to compute the similarity between the query sentence
query = 'Un homme mange des pâtes.'

# With all sentences in the corpus
corpus = ['Un homme mange de la nourriture.',
          'Un homme mange des biscuits',
          'Un homme mange un morceau de pain.',
          'La fille porte un bébé.',
          'Un homme monte à cheval.',
          'Une femme joue du violon.',
          'Deux hommes ont poussé des chariots à travers les bois.',
          'Un homme monte un cheval blanc sur un terrain clos.',
          'Un singe joue de la batterie.',
          "Un guépard court derrière sa proie."
          ]

# So we create the respective sentence combinations
sentence_combinations = [[query, corpus_sentence] for corpus_sentence in corpus]

# Compute the similarity scores for these combinations
similarity_scores = model_saved.predict(sentence_combinations)

# Sort the scores in decreasing order
sim_scores_argsort = reversed(np.argsort(similarity_scores))

# Print the scores

print("Query:", query)
for idx in sim_scores_argsort:
    print("{:.2f}\t{}".format(similarity_scores[idx], corpus[idx]))

Query: Un homme mange des pâtes.
0.68	Un homme mange de la nourriture.
0.52	Un homme mange des biscuits
0.49	Un homme mange un morceau de pain.
0.13	Deux hommes ont poussé des chariots à travers les bois.
0.12	Un homme monte à cheval.
0.11	Une femme joue du violon.
0.11	Un singe joue de la batterie.
0.11	Un homme monte un cheval blanc sur un terrain clos.
0.11	Un guépard court derrière sa proie.
0.11	La fille porte un bébé.


In [ ]:
 similarity_scores

array([0.68466586, 0.5196343 , 0.48736945, 0.111398  , 0.11502581,
       0.1126914 , 0.12724526, 0.11203343, 0.11221016, 0.11162288],
      dtype=float32)

In [ ]:
# """
# This example computes the score between a query and all possible
# sentences in a corpus using a Cross-Encoder for semantic textual similarity (STS).
# It output then the most similar sentences for the given query.
# """
# from sentence_transformers.cross_encoder import CrossEncoder
# import numpy as np

# # Pre-trained cross encoder
# # model = CrossEncoder('cross-encoder/stsb-distilroberta-base')
# model = CrossEncoder('cross-encoder/nli-deberta-v3-base')

# # We want to compute the similarity between the query sentence
# query = 'A man is eating pasta.'

# # With all sentences in the corpus
# corpus = ['A man is eating food.',
#           'A man is eating cookie',
#           'A man is eating a piece of bread.',
#           'The girl is carrying a baby.',
#           'A man is riding a horse.',
#           'A woman is playing violin.',
#           'Two men pushed carts through the woods.',
#           'A man is riding a white horse on an enclosed ground.',
#           'A monkey is playing drums.',
#           'A cheetah is running behind its prey.'
#           ]

# # So we create the respective sentence combinations
# sentence_combinations = [[query, corpus_sentence] for corpus_sentence in corpus]

# # Compute the similarity scores for these combinations
# similarity_scores = model.predict(sentence_combinations)
# print(similarity_scores)
# # Sort the scores in decreasing order
# # sim_scores_argsort = reversed(np.argsort(similarity_scores))

# # Print the scores
# # print("Query:", query)
# # for idx in sim_scores_argsort:
# #     print("{:.2f}\t{}".format(similarity_scores[idx], corpus[idx]))

[[-4.8803945   3.881933   -0.19317763]
 [ 6.1579623  -3.5691955  -2.233002  ]
 [ 6.6311836  -3.8391488  -2.3335292 ]
 [ 7.653007   -4.426881   -2.718175  ]
 [ 7.9211907  -4.424373   -3.0540853 ]
 [ 8.102759   -4.4515367  -3.2514944 ]
 [ 7.8335056  -4.166322   -3.2760768 ]
 [ 7.741339   -4.168262   -3.260732  ]
 [ 7.8938584  -4.299428   -3.2448034 ]
 [ 7.835708   -4.308041   -3.124826  ]]


In [ ]:
#stsb-multi-mt -- similarity score